In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
import cmocean.cm as cmo

import xarray as xr
import xgcm
from xhistogram.xarray import histogram
import xroms

from bichan_utils import *

import hvplot.xarray
import geoviews as gv

import cartopy.crs as ccrs
import cartopy.feature as cfeature
proj = ccrs.LambertConformal(central_longitude=-92, central_latitude=29)
coast_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='k', facecolor='0.8')
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

bal = plt.cm.get_cmap(cmo.balance)
newcolors = bal(np.linspace(0, 1, 256))
vals = list(range(256))
newcmp = ListedColormap(newcolors[vals[:110] + vals[-110:]])
# tiles = gv.tile_sources.ESRI

import warnings
warnings.filterwarnings("ignore")

/var/folders/qh/qtc973c11kz8h3wlry5lmvkh0000gn/T/ipykernel_35884/688150656.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  bal = plt.cm.get_cmap(cmo.balance)


In [3]:
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56957,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:56970,Total threads: 2
Dashboard: http://127.0.0.1:56973/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:56960,


In [4]:
ds = xr.open_zarr('./surface_child.zarr', 
                  consolidated=True)
ds, grid = xroms.roms_dataset(ds)
ds = add_derivatives(ds, grid)


# Figures

Some general definitions for the histogram analysis that will be used for all figures, so they are calculated on the same sub-domain over the same timeframe.

In [5]:
# time and space slices
tslice = {'ocean_time': slice('2010-6-15', '2010-6-20')}
ihslice = {'xi_rho': slice(50, 400),
          'eta_rho': slice(150, 350)}

# Horizontal area of cells
dA = 1.0/(ds.pm * ds.pn)

# Conditional to define only high, positive vorticity
conditional = (ds.zeta.isel(ihslice).sel(tslice) > 1.0) 
# conditional = True

In [6]:
timestrs = []
for t in ds.sel(tslice).ocean_time:
    timestr = t.dt.strftime('%Y-%m-%d %H:%M:%S').item()
    timestrs.append(timestr)
    
timestrs
    

['2010-06-15 00:00:00',
 '2010-06-15 01:00:00',
 '2010-06-15 02:00:00',
 '2010-06-15 03:00:00',
 '2010-06-15 04:00:00',
 '2010-06-15 05:00:00',
 '2010-06-15 06:00:00',
 '2010-06-15 07:00:00',
 '2010-06-15 08:00:00',
 '2010-06-15 09:00:00',
 '2010-06-15 10:00:00',
 '2010-06-15 11:00:00',
 '2010-06-15 12:00:00',
 '2010-06-15 13:00:00',
 '2010-06-15 14:00:00',
 '2010-06-15 15:00:00',
 '2010-06-15 16:00:00',
 '2010-06-15 17:00:00',
 '2010-06-15 18:00:00',
 '2010-06-15 19:00:00',
 '2010-06-15 20:00:00',
 '2010-06-15 21:00:00',
 '2010-06-15 22:00:00',
 '2010-06-15 23:00:00',
 '2010-06-16 00:00:00',
 '2010-06-16 01:00:00',
 '2010-06-16 02:00:00',
 '2010-06-16 03:00:00',
 '2010-06-16 04:00:00',
 '2010-06-16 05:00:00',
 '2010-06-16 06:00:00',
 '2010-06-16 07:00:00',
 '2010-06-16 08:00:00',
 '2010-06-16 09:00:00',
 '2010-06-16 10:00:00',
 '2010-06-16 11:00:00',
 '2010-06-16 12:00:00',
 '2010-06-16 13:00:00',
 '2010-06-16 14:00:00',
 '2010-06-16 15:00:00',
 '2010-06-16 16:00:00',
 '2010-06-16 17:

In [8]:
import numpy as np
from matplotlib.colors import ListedColormap
proj = ccrs.LambertConformal(central_longitude=-92, central_latitude=29)


for n, timestr in enumerate(timestrs):
    fig, axs = plt.subplots(4, 2, sharex=True, sharey=True, figsize=(14, 16.3),
                            subplot_kw={'projection': proj})

    fig.tight_layout()
    ax = axs[0,0]
    ds.salt.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=20, vmax=35, cmap=cmo.haline, ax=ax,
                                        cbar_kwargs={'label':r'Salinity [g kg$^{-1}$]'})
    add_box(ds, ax, ihslice, draw_labels=False, timestr=timestr)



    ax = axs[1,0]
    ds.zeta.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=-3, vmax=3, cmap=cmo.curl, ax=ax,
                                        cbar_kwargs={'label':r'Normalized vorticity [$\zeta f^{-1}$]'})
    add_box(ds, ax, ihslice, draw_labels=False, timestr=timestr)



    ax = axs[0,1]
    ds.sigma.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=0, vmax=3, cmap=cmo.turbid, ax=ax,
                                        cbar_kwargs={'label':r'Normalized total strain [$\sigma f^{-1}$]'})
    add_box(ds, ax, ihslice, draw_labels=False, timestr=timestr)

    ax = axs[1,1]
    ds.delta.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=-3, vmax=3, cmap=cmo.balance, ax=ax,
                                        cbar_kwargs={'label':r'Normalized divergence [$\delta f^{-1}$]'})
    add_box(ds, ax, ihslice, draw_labels=False, timestr=timestr)


    ax = axs[2,1]
    ds.du_cf.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=-3, vmax=3, cmap=cmo.balance, ax=ax,
                                        cbar_kwargs={'label':r'Normalized frontogenesis [$\frac{\partial \tilde{u}}{\partial \tilde{x}} f^{-1}$]'})
    add_box(ds, ax, ihslice, draw_labels=False, timestr=timestr)


    ax = axs[2,0]
    ds.tilde_sigma_n.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=-3, vmax=3, cmap=cmo.balance, ax=ax,
                                        cbar_kwargs={'label':r'Normalized effective strain [$\sigma_n f^{-1}$]'})
    add_box(ds, ax, ihslice, draw_labels=False, timestr=timestr)


    ax = axs[3,1]
    ds.du_af.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=-3, vmax=3, cmap=cmo.balance, ax=ax,
                                        cbar_kwargs={'label':r'Along-front velocity gradients [$\frac{\partial \tilde{v}}{\partial \tilde{y}} f^{-1}$]'})
    add_box(ds, ax, ihslice, draw_labels=False, timestr=timestr)

    new_cmap = cmo.tarn(np.linspace(0.1, 0.9, 256))
    new_cmap = ListedColormap(new_cmap)

    ax = axs[3,0]
    ds.strain_efficiency.sel(ocean_time=timestr).plot(x='lon_rho', y='lat_rho',
                                        transform=ccrs.PlateCarree(),
                                        vmin=-1.0, vmax=1.0, cmap=new_cmap, ax=ax,
                                        cbar_kwargs={'label':r'Strain efficiency [$\frac{\sigma_n}{\sigma}$]', 'ticks':[-1, 0, 1]})
    add_box(ds, ax, ihslice, draw_labels=True, timestr=timestr)

    plt.savefig('frame{n:04d}.png'.format(n=n), dpi=100, bbox_inches='tight')
    plt.close(fig)